In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments-average/'

In [3]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

In [4]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

## (TEST) Checking actual delay (from the start of the 3 epochs)

In [25]:
df_cumdelay = pd.read_pickle(DF_PATH + 'cumu-thrus/thrus_d450_a0_b1_c0.pkl')
df_delay0 = df_cumdelay.loc[60, :]

df_instdelay = df_cumdelay.diff()
df_instdelay.loc[60, :] = df_delay0
df_instdelay

level_1,"(1, 0, 0)","(1, 0, 1)","(1, 0, 2)","(1, 0, 3)"
t,,,,
60,20.444444,18.888889,18.000000,19.222222
120,45.555556,43.555556,43.888889,47.444444
180,69.111111,70.000000,70.777778,76.777778
240,94.222222,95.444444,95.666667,105.111111
300,119.000000,120.777778,121.666667,132.000000
360,124.555556,126.222222,128.555556,139.000000
420,124.111111,128.888889,130.555556,138.111111
480,130.333333,132.888889,133.777778,137.222222
540,132.555556,134.444444,139.444444,138.222222


In [11]:
list(df_instdelay.sum(axis=1)[[60, 240, 420, 600]])

[542.7079820926799, 6662.641460684859, 14974.698337857786, 19294.40414106777]

## (TEST) Checking expected delay (from the start of 3 epochs)

In [9]:
df_milp = pd.read_pickle(DF_PATH + 'ave_milp_results.pkl')
df_milp.head(10)

,demand,new_model,alpha,beta,gamma,model_type,epoch,runtime,delay,throughput,objective_value
0,450,False,0.0,0.0,0.0,Parent model,0,370.309661,2313.000000,79.500000,79.500000
1,450,False,0.0,0.0,0.0,Parent model,1,79.655836,6809.105485,127.363221,127.363221
2,450,False,0.0,0.0,0.0,Parent model,2,76.947723,9181.654008,128.917018,128.917018
3,450,False,0.0,0.0,0.0,Parent model,3,81.051107,12968.939522,129.139241,129.139241
4,450,False,0.0,0.0,1.0,Flow priority,0,573.475265,3265.265823,52.278481,-2055.801688
5,450,False,0.0,0.5,0.5,Throughput-Flow priority,0,293.788045,3220.430380,54.000000,-1128.895162
6,450,False,0.0,1.0,0.0,Throughput priority,0,268.785380,3500.000000,55.000000,-205.729167
7,450,True,0.0,0.0,1.0,Flow priority,1,0.506242,8693.330520,64.900141,-1599.572199
8,450,True,0.0,0.0,1.0,Flow priority,2,0.512985,11508.604782,66.756681,-1471.155649
9,450,True,0.0,0.0,1.0,Flow priority,3,0.549048,14773.066104,67.153305,-1186.626817


In [12]:
df_milp[(df_milp.demand == 450) & (df_milp.model_type == 'Throughput priority')].delay.values

array([ 3500.        ,  9198.29817158, 12121.89029536, 15735.56399437])

## Combining expected and actual results

In [17]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [18]:
delay_dfs = []
throughput_dfs = []

df_milp = pd.read_pickle(DF_PATH + 'ave_milp_results.pkl')
df_delay = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])
df_thru = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])

for demand in _demands:
    for weights in _weights:

        # Get actual values
        
        df_cumdelay_tmp = pd.read_pickle(DF_PATH + 'cumu-delays/delays_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instdelay_tmp = df_cumdelay_tmp.diff()
        df_instdelay_tmp.loc[60, :] = df_cumdelay_tmp.loc[60, :]
        actual_delay = list(df_instdelay_tmp.sum(axis=1)[[60, 240, 420, 600]])
        
        df_cumthru_tmp = pd.read_pickle(DF_PATH + 'cumu-thrus/thrus_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instthru_tmp = df_cumthru_tmp.diff()
        df_instthru_tmp.loc[60, :] = df_cumthru_tmp.loc[60, :]
        actual_thru = list(df_instthru_tmp.sum(axis=1)[[60, 240, 420, 600]])

        # Get expected values
        
        expected_delay = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].delay)
        expected_thru = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].throughput)
        
        # Combine into a dataframe
        
        for i in range(4):
            df_delay.loc[-1] = [demand, _model_type[weights], expected_delay[i], actual_delay[i], i]
            df_thru.loc[-1] = [demand, _model_type[weights], expected_thru[i], actual_thru[i], i]
            df_delay.index += 1
            df_thru.index += 1

In [19]:
df_delay.to_pickle(DF_PATH + "delay_comparison.pkl")
df_thru.to_pickle(DF_PATH + "throughput_comparison.pkl")

In [6]:
df_delay = pd.read_pickle(DF_PATH + "delay_comparison.pkl")
df_thru = pd.read_pickle(DF_PATH + "throughput_comparison.pkl")
df_thru

,demand,model_type,expected,actual,epoch
127,450,Delay priority,51.278481,76.222222,0
126,450,Delay priority,67.248945,399.555556,1
125,450,Delay priority,68.057665,538.666667,2
124,450,Delay priority,68.724332,561.555556,3
123,450,Throughput priority,55.000000,76.555556,0
122,450,Throughput priority,67.375527,390.444444,1
121,450,Throughput priority,68.946554,521.666667,2
120,450,Throughput priority,69.946554,554.888889,3
119,450,Flow priority,52.278481,78.333333,0
118,450,Flow priority,64.900141,400.444444,1
